<a href="https://colab.research.google.com/github/Chatrawit/main/blob/main/Spark_ML_Regression_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=a2a4d268083b49abeea787d3d6892dbd2e2e7fa16a06ac7f055ebce4731eefed
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
spark = SparkSession \
    .builder \
    .appName("ML_test") \
    .getOrCreate()

# Create Spark Context

In [ ]:
sc = spark.sparkContext


sqlContext = SQLContext(sc)

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
from google.colab import files


uploaded = files.upload()


Saving daily_weather.csv to daily_weather.csv


In [ ]:
file='daily_weather.csv'

df = sqlContext.read.load(file,
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

In [ ]:
from pyspark.ml import Pipeline


#  Regression

In [ ]:
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline


In [ ]:
df = df.drop('number')
df = df.na.drop() 

In [ ]:
featureColumns =df.columns[:-2]

In [ ]:
(trainingData, testData) = df.randomSplit([0.8,0.2], seed = 13234 )

In [ ]:


df=df.withColumnRenamed("relative_humidity_3pm","label")
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
lr = LinearRegression()
pipeline = Pipeline(stages=[assembler, lr])

In [ ]:
model = pipeline.fit(trainingData)


In [ ]:
prediction = model.transform(testData)


In [ ]:
prediction.show()

+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+------------------+--------------------+------------------+
| air_pressure_9am|      air_temp_9am|avg_wind_direction_9am|avg_wind_speed_9am|max_wind_direction_9am|max_wind_speed_9am|rain_accumulation_9am|rain_duration_9am|relative_humidity_9am|             label|            features|        prediction|
+-----------------+------------------+----------------------+------------------+----------------------+------------------+---------------------+-----------------+---------------------+------------------+--------------------+------------------+
|911.2700000000134| 59.82800000000221|    218.30000000000004| 2.259309399999348|    235.89999999999992|2.9751301999997906|                  0.0|              0.0|    86.73000000000106| 54.06000000000101|[911.270000000013...| 69.31446556132505|
|912.3000000000123| 74.5

In [ ]:
prediction.select('label','prediction').show()

+------------------+------------------+
|             label|        prediction|
+------------------+------------------+
| 54.06000000000101| 69.31446556132505|
| 56.60000000000103| 47.05583869897373|
|             78.01| 66.25235474155261|
| 58.28000000000014| 63.82424741217437|
| 91.34000000000015| 74.26974687399843|
| 80.87000000000023| 81.48210625090906|
| 47.99000000000034| 56.92027772822348|
| 88.55999999999996| 71.47105587986016|
|13.690000000000074|34.519697913741766|
|56.050000000000495| 54.16776803907851|
|48.890000000001024|  58.7124650268629|
|54.180000000000454| 49.08777466543961|
| 73.08000000000013| 81.48663875405919|
|23.634849837346575|33.079644752918284|
| 90.65000000000073| 70.87944181724697|
| 66.63000000000122| 57.47629948941585|
|31.600000000001053| 36.56941023832405|
| 45.55000000000043| 35.76529207805106|
| 33.88000000000106| 38.68256024704624|
| 58.25000000000021| 57.61541429042745|
+------------------+------------------+
only showing top 20 rows



In [ ]:
sc.stop()